# 🚚 Fase 05: Logistics & Delivery Performance
---
**Objetivo**: Analizar la eficiencia logística y el cumplimiento de promesas de entrega.

## 1. Carga de Datos
* Objetivo: Preparar el entorno y cargar los datasets necesarios para el análisis logístico.
* Datasets: orders_cleaned.csv y order_items_totals.csv.

In [3]:
import pandas as pd
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
path_processed = "../data/processed"
# Convertir strings a objetos `datetime`:

# Carga de datos
orders = pd.read_csv(f'{path_processed}/orders_cleaned.csv',
                        parse_dates=['order_purchase_timestamp', 
                                      'order_approved_at', 
                                      'order_delivered_carrier_date', 
                                      'order_delivered_customer_date', 
                                      'order_estimated_delivery_date'])
customers = pd.read_csv(f"{path_processed}/customers_cleaned.csv")


In [5]:
orders.info()
orders.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 8   delivery_time_days             96476 non-null  float64       
 9   delta_estimated_days           96476 non-null  float64       
dtypes: datetime64[ns](5), float64(2), object(3)
memory usage: 7.6+ MB


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delta_estimated_days
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8.0,-8.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13.0,-6.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9.0,-18.0


In [6]:
customers.info()
customers.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP


## 2. Segmentación del Desempeño Logístico
* **Objetivo:** Clasificar los pedidos según el cumplimiento de la promesa de entrega.
* **Lógica:** * Pedidos con `delta_estimated_days` > 0: **Retrasados**.
    * Pedidos con `delta_estimated_days` <= 0: **A tiempo / Anticipados**.

In [8]:
# 1. Aseguro la creación de la categoría basada en el delta que ya tengo
orders['delivery_status'] = orders['delta_estimated_days'].apply(
    lambda x: 'Retrasado' if x > 0 else 'A tiempo / Anticipado'
)

# 2. Ahora sí, cuento cuántos hay de cada categoría (Status)
resumen_logistica = orders['delivery_status'].value_counts(normalize=True) * 100

print("--- Análisis de Cumplimiento de Entrega ---")
print(resumen_logistica)

# 3. Confirmo el número absoluto de retrasos
total_retrasos = (orders['delivery_status'] == 'Retrasado').sum()
print(f"\nTotal de pedidos con retraso: {total_retrasos}")

--- Análisis de Cumplimiento de Entrega ---
delivery_status
A tiempo / Anticipado    93.428264
Retrasado                 6.571736
Name: proportion, dtype: float64

Total de pedidos con retraso: 6535


#### 📝 Conclusión del análisis de la segmentación del desempeño logístico:
Insight de Entrega: El sistema logístico de Olist cumple con la promesa de entrega en el 93.4 % de los casos. Sin embargo, existe un 6.5 % de pedidos (6,535 órdenes) que llegan después de la fecha estimada, lo cual es un volumen crítico que impacta directamente en la satisfacción del cliente y posibles reclamos.

## 3. Análisis Geográfico de Retrasos
* **Objetivo:** Identificar los estados con mayor índice de incumplimiento.
* **Pregunta de Negocio:** ¿El retraso es culpa de la infraestructura de ciertos estados o es un problema global?
* **Técnica:** Agrupación por estado y cálculo del porcentaje de pedidos retrasados.

In [11]:
orders.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delta_estimated_days,delivery_status
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8.0,-8.0,A tiempo / Anticipado
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13.0,-6.0,A tiempo / Anticipado
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9.0,-18.0,A tiempo / Anticipado


In [12]:
orders[(orders['delivery_status'] == 'Retrasado')].head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delta_estimated_days,delivery_status
20,203096f03d82e0dffbc41ebc2e2bcfb7,d2b091571da224a1b36412c18bc3bbfe,delivered,2017-09-18 14:31:30,2017-09-19 04:04:09,2017-10-06 17:50:03,2017-10-09 22:23:46,2017-09-28,21.0,11.0,Retrasado
25,fbf9ac61453ac646ce8ad9783d7d0af6,3a874b4d4c4b6543206ff5d89287f0c3,delivered,2018-02-20 23:46:53,2018-02-22 02:30:46,2018-02-26 22:25:22,2018-03-21 22:03:54,2018-03-12,28.0,9.0,Retrasado
41,6ea2f835b4556291ffdc53fa0b3b95e8,c7340080e394356141681bd4c9b8fe31,delivered,2017-11-24 21:27:48,2017-11-25 00:21:09,2017-12-13 21:14:05,2017-12-28 18:59:23,2017-12-21,33.0,7.0,Retrasado


In [26]:
# Voy a cruzar los pedidos con los clientes para obtener el estado
query_delays_geo = """
SELECT
    customer_state,
    SUM(
        CASE
            WHEN delivery_status = 'Retrasado' THEN 1
            ELSE 0
        END
    ) AS delayed_orders,
    ROUND(SUM(
        CASE
            WHEN delivery_status = 'Retrasado' THEN 1
            ELSE 0
        END
    ) / COUNT(*) * 100, 2)  AS porcentage_delayed  
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE order_status = 'delivered'
GROUP BY 1
ORDER BY porcentage_delayed DESC
LIMIT 10;
"""
duckdb.sql(query_delays_geo).show()

┌────────────────┬────────────────┬────────────────────┐
│ customer_state │ delayed_orders │ porcentage_delayed │
│    varchar     │     int128     │       double       │
├────────────────┼────────────────┼────────────────────┤
│ AL             │             85 │              21.41 │
│ MA             │            125 │              17.43 │
│ SE             │             51 │              15.22 │
│ PI             │             66 │              13.87 │
│ CE             │            176 │              13.76 │
│ RR             │              5 │               12.2 │
│ BA             │            396 │              12.16 │
│ RJ             │           1495 │              12.11 │
│ PA             │            106 │              11.21 │
│ ES             │            214 │              10.73 │
├────────────────┴────────────────┴────────────────────┤
│ 10 rows                                    3 columns │
└──────────────────────────────────────────────────────┘



In [36]:
# Calculo el promedio nacional de retraso para comparar
national_average = (orders['delivery_status'] == 'Retrasado').mean() * 100
print(f"El porcentaje real de retraso nacional es: {national_average:.2f}%")

El porcentaje real de retraso nacional es: 6.57%


#### 📝 Insight de Volumen vs. Proporción: 
Existe una disparidad geográfica importante. Mientras que estados remotos como **RR** tienen un porcentaje alto (**12.2%**) con solo **5 retrasos** debido a su bajo volumen, el caso de **RJ** es alarmante: mantiene un porcentaje similar (**12.1%**) pero con un volumen masivo de **1,495 retrasos**, lo que sugiere problemas de seguridad o saturación urbana que difieren de los problemas de distancia del Norte. Esto indica que el problema en RJ es estructural y de alto impacto para la empresa.